In [1]:
from sklearn import datasets
from scipy.io.arff import loadarff
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd


In [2]:
forest = datasets.fetch_covtype()

In [3]:
type(forest)

sklearn.datasets.base.Bunch

In [4]:
X, X_test, y, y_test = train_test_split(
    forest.data, forest.target == 2, test_size=0.3, random_state=1234)

In [5]:
print(X.shape, y.shape, X_test.shape, y_test.shape)

(406708, 54) (406708,) (174304, 54) (174304,)


## Maintenant un peu de xgboost!

### D'abord un tout petit

In [13]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 2
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.798104	train-auc:0.797500
[1]	eval-auc:0.835330	train-auc:0.836546


In [31]:
X_test[0, [0,25,35,15]]

array([ 2687.,     0.,     0.,     0.])

In [37]:
leaves = bst.predict(dtest, pred_leaf=True)
y = bst.predict(dtest)
leaves[:3], y[:3], inv_logit(0.388039+ 0.354413)

/home/yannick/bin/anaconda3/envs/py35/lib/python3.5/site-packages/xgboost/core.py:840: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  preds = preds.reshape(nrow, preds.size / nrow)


(array([[39, 40],
        [39, 40],
        [39, 40]], dtype=int32),
 array([ 0.67753184,  0.67753184,  0.67753184], dtype=float32),
 0.67753180825889836)

In [36]:
leaves[400], y[400], X_test[400, [0,7,3]]

(array([51, 53], dtype=int32), 0.50070846, array([ 3126.,   254.,   283.]))

In [18]:
import numpy as np
def logit(p):
    return np.log(p) - np.log(1 - p)

def inv_logit(p):
    return np.exp(p) / (1 + np.exp(p))

(0.67753180825889836, 0.74245214527986758)

In [15]:
bst.dump_model("bst_2_trees")
! cat bst_2_trees

booster[0]:
0:[f0<3073.5] yes=1,no=2,missing=1
	1:[f0<2497.5] yes=3,no=4,missing=3
		3:[f0<2436.5] yes=7,no=8,missing=7
			7:[f5<659.5] yes=15,no=16,missing=16
				15:[f0<2220.5] yes=31,no=32,missing=31
					31:leaf=-0.98394
					32:leaf=-0.793315
				16:[f9<1375] yes=33,no=34,missing=33
					33:leaf=-0.995588
					34:leaf=-0.891282
			8:[f23<-1.00136e-05] yes=17,no=18,missing=17
				17:[f10<-1.00136e-05] yes=35,no=36,missing=35
					35:leaf=-0.822691
					36:leaf=0.388889
				18:[f3<205.5] yes=37,no=38,missing=37
					37:leaf=-0.171043
					38:leaf=-0.758065
		4:[f25<-1.00136e-05] yes=9,no=10,missing=9
			9:[f35<-1.00136e-05] yes=19,no=20,missing=19
				19:[f15<-1.00136e-05] yes=39,no=40,missing=39
					39:leaf=0.388039
					40:leaf=-0.642651
				20:[f0<2807.5] yes=41,no=42,missing=41
					41:leaf=0.789916
					42:leaf=-0.458464
			10:[f9<6667] yes=21,no=22,missing=22
				21:[f5<4885.5] yes=43,no=44,missing=44
					43:leaf=0.868075
					44:leaf=0.458409
				22:[f9<6873.5] yes=45,no=46

In [6]:
import xgboost as xgb

dtrain = xgb.DMatrix(X, label=y)
dtest  = xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]

param = {
    'bst:max_depth': 5, 
    'bst:eta': 0.5, 
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 201
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.798104	train-auc:0.797500
[40]	eval-auc:0.922583	train-auc:0.925691
[80]	eval-auc:0.939016	train-auc:0.943107
[120]	eval-auc:0.953977	train-auc:0.958951
[160]	eval-auc:0.961968	train-auc:0.967478
[200]	eval-auc:0.967300	train-auc:0.973411


In [7]:
param = {
    'bst:max_depth': 9, 
    'bst:eta': 0.6,
    'colsample_bytree': 0.74,
    'subsample': 0.66,
    'silent': True, 
    'verbose': 0,
    'objective': 'binary:logistic',
    'nthread': 4,
    'eval_metric': 'auc'
}

num_round = 201
bst = xgb.train(param,
                dtrain,
                num_round,
                evallist,
                verbose_eval    = 40
               )


[0]	eval-auc:0.744344	train-auc:0.747410
[40]	eval-auc:0.975541	train-auc:0.983620
[80]	eval-auc:0.986208	train-auc:0.994752
[120]	eval-auc:0.990154	train-auc:0.998182
[160]	eval-auc:0.992087	train-auc:0.999415
[200]	eval-auc:0.992860	train-auc:0.999795
